# 05 Comparative Visualization

This notebook generates scatter plots and other visualizations, comparing various dependency size metrics and linguistic factors. It uses the standardized plotting style (connecting lines to labels) for all scatter plots.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from importlib import reload

# Custom modules
import data_utils
import plotting

# Reload plotting to ensure latest style
reload(plotting)

%matplotlib inline

# Configuration
DATA_DIR = "data"
PLOTS_DIR = "plots"

## 1. Load Data

In [2]:
# Load metadata and metrics
metadata = data_utils.load_metadata(os.path.join(DATA_DIR, 'metadata.pkl'))
langNames = metadata['langNames']
langnameGroup = metadata['langnameGroup']
group_to_color = metadata['appearance_dict']

with open(os.path.join(DATA_DIR, 'all_langs_average_sizes_filtered.pkl'), 'rb') as f:
    all_langs_average_sizes_filtered = pickle.load(f)

# Load Pre-computed factors (e.g. HCS)
if os.path.exists(os.path.join(DATA_DIR, 'hcs_factors.csv')):
    hcs_df = pd.read_csv(os.path.join(DATA_DIR, 'hcs_factors.csv'))
else:
    print("Warning: hcs_factors.csv not found. Please run 04_data_processing using compute_factors.")
    hcs_df = pd.DataFrame()

if os.path.exists(os.path.join(DATA_DIR, 'vo_vs_hi_scores.csv')):
    vo_hi_df = pd.read_csv(os.path.join(DATA_DIR, 'vo_vs_hi_scores.csv'))
else:
    print("Warning: vo_vs_hi_scores.csv not found.")
    vo_hi_df = pd.DataFrame()

print(f"Loaded data for {len(all_langs_average_sizes_filtered)} languages")

Loaded metadata from data/metadata.pkl
Loaded data for 185 languages


## 2. MAL Plots (Mean Average Length)

In [3]:
# Example MAL plot
# plotting.plot_dependency_sizes(
#     'right_1_totright_2', 'right_2_totright_2',
#     'MAL',
#     all_langs_average_sizes_filtered,
#     lambda x: True,
#     langNames, langnameGroup,
#     palette=group_to_color
# )

# producing loads of plots

this takes > 12 minutes

In [4]:
# Prepare Data for Filters (without generating plots)
import pandas as pd
import plotting
from importlib import reload
from tqdm.notebook import tqdm
# Reload plotting to get the latest sns.scatterplot changes and silent mode
reload(plotting)

print("Preparing language filters...")

# 1. Create mapping for Head Initiality & VO Score
# ---------------------------
if 'head_initiality_score' in vo_hi_df.columns:
    vo_hi_df['head_initiality'] = vo_hi_df['head_initiality_score']

join_col_hi = 'language' if 'language' in vo_hi_df.columns else 'language_code'
lang_to_head_init = vo_hi_df.set_index(join_col_hi)['head_initiality'].to_dict()

# Load vo_type classification (tripartite: VO/OV/NDO)
if 'vo_type' in vo_hi_df.columns:
    lang_to_vo_type = vo_hi_df.set_index(join_col_hi)['vo_type'].to_dict()
else:
    # Fallback: compute from vo_score if vo_type not available
    if 'vo_score' in vo_hi_df.columns:
        lang_to_vo = vo_hi_df.set_index(join_col_hi)['vo_score'].to_dict()
        lang_to_vo_type = {}
        for lang, score in lang_to_vo.items():
            if score is not None and not pd.isna(score):
                if score > 0.666:
                    lang_to_vo_type[lang] = 'VO'
                elif score < 0.333:
                    lang_to_vo_type[lang] = 'OV'
                else:
                    lang_to_vo_type[lang] = 'NDO'
            else:
                lang_to_vo_type[lang] = None
    else:
        lang_to_vo_type = {}
        print("Warning: Neither 'vo_type' nor 'vo_score' found in vo_hi_df. VO/OV/NDO plots will be skipped.")

# Calculate Lists of Languages for Parallel Processing
indo_euro_langs = {l for l in all_langs_average_sizes_filtered if langnameGroup.get(langNames.get(l, l), 'Other') == 'Indo-European'}
non_indo_euro_langs = {l for l in all_langs_average_sizes_filtered if langnameGroup.get(langNames.get(l, l), 'Other') != 'Indo-European'}

head_init_langs = {l for l in all_langs_average_sizes_filtered if lang_to_head_init.get(l, 0) > 0.5}
head_final_langs = {l for l in all_langs_average_sizes_filtered if lang_to_head_init.get(l, 0) <= 0.5}

# Tripartite word order classification
vo_langs = {l for l in all_langs_average_sizes_filtered if lang_to_vo_type.get(l) == 'VO'}
ov_langs = {l for l in all_langs_average_sizes_filtered if lang_to_vo_type.get(l) == 'OV'}
ndo_langs = {l for l in all_langs_average_sizes_filtered if lang_to_vo_type.get(l) == 'NDO'}

# Prepare factors dataframe
if not hcs_df.empty:
    merge_col = 'language' if 'language' in hcs_df.columns else 'language_code'
    if merge_col in hcs_df.columns:
        # Use the same join column logic
        join_col_hi = 'language' if 'language' in vo_hi_df.columns else 'language_code'
        head_init_df = vo_hi_df[[join_col_hi, 'head_initiality']].rename(columns={join_col_hi: merge_col})
        
        all_factors_df = pd.merge(hcs_df, head_init_df, on=merge_col, how='inner')
        all_factors_df['group'] = all_factors_df[merge_col].map(lambda x: langnameGroup.get(langNames.get(x, x), 'Other'))
        all_factors_df['language_name'] = all_factors_df[merge_col].map(lambda x: langNames.get(x, x))
    else:
        print("Warning: Merge col mismatch.")
        all_factors_df = pd.DataFrame()
else:
    print("Warning: hcs_factors.csv not available.")
    all_factors_df = pd.DataFrame()

print(f"\n✅ Filters prepared:")
print(f"  Indo-European: {len(indo_euro_langs)}")
print(f"  Non-Indo-European: {len(non_indo_euro_langs)}")
print(f"  Head-Initial: {len(head_init_langs)}")
print(f"  Head-Final: {len(head_final_langs)}")
print(f"  VO: {len(vo_langs)}")
print(f"  OV: {len(ov_langs)}")
print(f"  NDO: {len(ndo_langs)}")

Preparing language filters...

✅ Filters prepared:
  Indo-European: 91
  Non-Indo-European: 94
  Head-Initial: 66
  Head-Final: 119
  VO: 79
  OV: 69
  NDO: 37


In [5]:
# Generate all plots (this cell takes > 12 minutes)
print("Starting generation of ALL plot subsets...")

# 1. Generate Head-Init vs Factors Plots
# --------------------------------------
print("\n--- Generating Head-Init vs Factors Plots ---")
if not all_factors_df.empty:
    plotting.plot_head_initiality_vs_factors(all_factors_df, group_to_color)
else:
    print("Skipping (no factors data)")

# 2. Generate Scatter Plot Subsets (Batch)
# ----------------------------------------
subsets = [
    ("Head Final Languages", "headFinal-", head_final_langs),
    ("Head Initial Languages", "headInit-", head_init_langs),
    ("Indo-European Languages", "IE-", indo_euro_langs),
    ("Non-Indo-European Languages", "noIE-", non_indo_euro_langs),
    ("VO (Verb-Object) Languages", "VO-", vo_langs),
    ("OV (Object-Verb) Languages", "OV-", ov_langs),
    ("NDO (No Dominant Order) Languages", "NDO-", ndo_langs)
]

print("\nGenerating scatter plots for subsets...")
# Use tqdm for the overall progress
pbar = tqdm(subsets, desc="Subsets")
for name, prefix, lang_set in pbar:
    pbar.set_postfix_str(f"Processing {name}")
    if not lang_set:
        # print(f"Skipping {name} (no languages found)")
        continue
        
    plotting.plot_all(
        all_langs_average_sizes_filtered, 
        langNames, 
        langnameGroup, 
        filter_lang=lang_set,
        langname_group_or_genus=langnameGroup, 
        folderprefix=prefix, 
        palette=group_to_color, 
        parallel=True
    )

print("\n✅ All plots generated!")

Starting generation of ALL plot subsets...

--- Generating Head-Init vs Factors Plots ---
Generating 9 scatter plots (3 versions per factor)...
Total tasks: 9
Using 80 CPU cores for parallel processing


Generating Head-Init Plots:   0%|          | 0/9 [00:00<?, ?it/s]


✅ Completed 6 scatter plots in plots/head_init_vs_factors/

Generating scatter plots for subsets...


Subsets:   0%|          | 0/7 [00:00<?, ?it/s]

_________________________ plotting all ___________________________ headFinal-
Total plots to generate: 60
Using 79 CPU cores for parallel processing


...

Generating plots:   0%|          | 0/60 [00:00<?, ?it/s]

/bigstorage/kim/ragenv/lib/python3.10/site-packages/statsmodels/robust/robust_linear_model.py:289: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  warnings.warn('Estimated scale is 0.0 indicating that the most'
/bigstorage/kim/ragenv/lib/python3.10/site-packages/statsmodels/robust/robust_linear_model.py:289: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  warnings.warn('Estimated scale is 0.0 indicating that the most'
/bigstorage/kim/ragenv/lib/python3.10/site-packages/statsmodels/robust/robust_linear_model.py:289: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  warnings.warn('Estimated scale is 0.0 indicating that the most'
/bigstorage/kim/ragenv/lib/python3.10/site-packages/statsmodels/robust/robust_linear_model.py:289: ConvergenceWarning: Es

\n✅ Completed 60 plots
_________________________ plotting all ___________________________ NDO-
Total plots to generate: 60
Using 79 CPU cores for parallel processing


Generating plots:   0%|          | 0/60 [00:00<?, ?it/s]

\n✅ Completed 60 plots

✅ All plots generated!


## VS vs VO Analysis

Analyze the relationship between Subject-Verb order (VS score) and Verb-Object order (VO score) across different language groups.

**Note:** The `sv_score` column measures the proportion of subjects that appear **after** the verb (VS order), so:
- High score (>66.6%) = VS languages (verb-subject)
- Low score (<33.3%) = SV languages (subject-verb)

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns

# Debug: Check what columns we have
print("Available columns in vo_hi_df:")
print(vo_hi_df.columns.tolist())
print()

# Ensure join_col_hi is defined
if 'join_col_hi' not in dir():
    join_col_hi = 'language' if 'language' in vo_hi_df.columns else 'language_code'
    print(f"Setting join_col_hi to: {join_col_hi}")

# Create VS type mapping if not present
# Note: sv_score measures subjects AFTER verb (VS order), so high score = VS
if 'sv_type' in vo_hi_df.columns:
    lang_to_vs_type = vo_hi_df.set_index(join_col_hi)['sv_type'].to_dict()
else:
    # Fallback: compute from sv_score if sv_type not available
    if 'sv_score' in vo_hi_df.columns:
        lang_to_vs = vo_hi_df.set_index(join_col_hi)['sv_score'].to_dict()
        lang_to_vs_type = {}
        for lang, score in lang_to_vs.items():
            if score is not None and not pd.isna(score):
                if score > 0.666:
                    lang_to_vs_type[lang] = 'VS'  # Most subjects after verb
                elif score < 0.333:
                    lang_to_vs_type[lang] = 'SV'  # Most subjects before verb
                else:
                    lang_to_vs_type[lang] = 'NDO_SV'
            else:
                lang_to_vs_type[lang] = None
    else:
        lang_to_vs_type = {}
        print("⚠️ WARNING: Neither 'sv_type' nor 'sv_score' found in vo_hi_df.")
        print("Please ensure you have:")
        print("1. Re-run cell 9 in notebook 02 (conll_processing.get_all_stats_parallel)")
        print("2. Re-run cell 4 in this notebook (reload vo_vs_hi_scores.csv)")

# Calculate VS classification sets
sv_langs = {l for l in all_langs_average_sizes_filtered if lang_to_vs_type.get(l) == 'SV'}
vs_langs = {l for l in all_langs_average_sizes_filtered if lang_to_vs_type.get(l) == 'VS'}
ndo_sv_langs = {l for l in all_langs_average_sizes_filtered if lang_to_vs_type.get(l) == 'NDO_SV'}

print(f"SV languages (subjects before verb): {len(sv_langs)}")
print(f"VS languages (subjects after verb): {len(vs_langs)}")
print(f"NDO_SV languages: {len(ndo_sv_langs)}")

Available columns in vo_hi_df:
['vo_score', 'vo_type', 'sv_score', 'sv_type', 'head_initiality_score', 'vo_count', 'sv_count', 'total_deps', 'vo_right', 'sv_right', 'all_right', 'language_code', 'language_name', 'group', 'head_initiality']

SV languages (subjects before verb): 158
VS languages (subjects after verb): 10
NDO_SV languages: 16


### Generate VS vs VO Scatterplots for Different Language Groups

In [11]:
# Generate VS vs VO scatterplots for different language groups (in parallel)
# Reload plotting to get the new function
reload(plotting)

# Define all language filters for batch processing
language_filters = [
    ("VS vs VO Scores - All Languages", "all_", set(all_langs_average_sizes_filtered.keys())),
    ("VS vs VO Scores - VO Languages (VO score > 66.6%)", "vo_", vo_langs),
    ("VS vs VO Scores - OV Languages (VO score < 33.3%)", "ov_", ov_langs),
    ("VS vs VO Scores - NDO Languages (33.3% ≤ VO score ≤ 66.6%)", "ndo_", ndo_langs),
    ("VS vs VO Scores - Indo-European Languages", "ie_", indo_euro_langs),
    ("VS vs VO Scores - Non-Indo-European Languages", "noie_", non_indo_euro_langs),
]

# Generate all plots in parallel
results = plotting.plot_sv_vs_vo_scatter_batch(
    vo_hi_df, 
    language_filters, 
    group_to_color, 
    plots_dir=PLOTS_DIR,
    parallel=True
)

Generating 6 VS vs VO scatterplots...
Using 79 CPU cores for parallel processing


Generating VS vs VO plots:   0%|          | 0/6 [00:00<?, ?it/s]


✅ Completed 6 VS vs VO plots

VS vs VO Analysis Statistics

VS vs VO Scores - All Languages:
  Total languages: 184
  Mean VS score: 0.180
  Mean VO score: 0.493
  Correlation: 0.436
  VO & VS (both >66.6%): 10
  VO & SV (VO>66.6%, VS<33.3%): 62
  OV & VS (VO<33.3%, VS>66.6%): 0
  OV & SV (both <33.3%): 65

VS vs VO Scores - VO Languages (VO score > 66.6%):
  Total languages: 78
  Mean VS score: 0.261
  Mean VO score: 0.858
  Correlation: 0.031
  VO & VS (both >66.6%): 10
  VO & SV (VO>66.6%, VS<33.3%): 62
  OV & VS (VO<33.3%, VS>66.6%): 0
  OV & SV (both <33.3%): 0

VS vs VO Scores - OV Languages (VO score < 33.3%):
  Total languages: 69
  Mean VS score: 0.066
  Mean VO score: 0.066
  Correlation: 0.360
  VO & VS (both >66.6%): 0
  VO & SV (VO>66.6%, VS<33.3%): 0
  OV & VS (VO<33.3%, VS>66.6%): 0
  OV & SV (both <33.3%): 65

VS vs VO Scores - NDO Languages (33.3% ≤ VO score ≤ 66.6%):
  Total languages: 37
  Mean VS score: 0.224
  Mean VO score: 0.517
  Correlation: 0.489
  VO & VS (b